In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info = True, as_supervised = True)

In [12]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [13]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_validation_samples



<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [14]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [15]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image = image/255.
    return image,label

In [16]:
scaled_train_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [22]:
BUFFERSIZE = 10000
BATCHSIZE = 50
NUM_EPOCHS = 5

input_size = 784
output_size = 10
hidden_layer_size = 200


shuffle_train_validation_data = scaled_train_validation_data.shuffle(BUFFERSIZE)


validation_data = shuffle_train_validation_data.take(num_validation_samples)
train_data  = shuffle_train_validation_data.skip(num_validation_samples)

train_data = train_data.batch(BATCHSIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                            tf.keras.layers.Dense(output_size, activation = 'softmax')
                            ])


model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])


model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2 )

Epoch 1/5
1080/1080 - 5s - loss: 0.2187 - accuracy: 0.9335 - val_loss: 0.1086 - val_accuracy: 0.9667
Epoch 2/5
1080/1080 - 4s - loss: 0.0944 - accuracy: 0.9707 - val_loss: 0.0799 - val_accuracy: 0.9752
Epoch 3/5
1080/1080 - 4s - loss: 0.0681 - accuracy: 0.9785 - val_loss: 0.0640 - val_accuracy: 0.9803
Epoch 4/5
1080/1080 - 4s - loss: 0.0542 - accuracy: 0.9829 - val_loss: 0.0578 - val_accuracy: 0.9818
Epoch 5/5
1080/1080 - 4s - loss: 0.0431 - accuracy: 0.9866 - val_loss: 0.0580 - val_accuracy: 0.9828


In [23]:
model.evaluate(test_data)

1/1 [==============================] - 0s 245ms/step - loss: 0.0862 - accuracy: 0.9773


[0.0861651748418808, 0.9772999882698059]